# GWT with GraphQL

In [1]:
from graphene import Boolean, Field, Int, InputObjectType, List, Mutation, NonNull, ObjectType, String, Schema
import json

### For a stored computed column JSON

* The computed column GraphQL object can follow this structure
* The _given_, _using_, _when_, and _action_ strings are implementing actual python dataframe syntax
* The JSON, CSV, or SQL data must be converted to a single dataframe called "\_data"
* Thereafter, the _using_ syntax will use the \_given_data dataframe, then \_when will use \_using_data, and _action_ will use the \_when_data

```someComputedCol {
    computeAction(dslInput: 
    {
        given : "_data.loc[data['UUID'] == '{cust_edi_uuid}']"
        using : "_given_data[['UUID', 'ETA', 'ETD', 'CONSOL', 'PORT']]"
        when : "_using_data.loc[_using_df['ETA'] > pd.to_datetime(date.today())]"
        action : "str(_when_data['ETA'].item())+' '+str(_when_data['CONSOL'].item())+' '+str(_when_data['PORT'].item())"
        })
        {
        gwt {
            given
            using
            when
            action
        }
        returnVal
    }
}```

## Dummy dataset & data types
The dataset will contain the data and the types for each of the columns.

### Computed Columns

In [146]:
jsonData = {
    "data": [{
        "UUID": "X1234",
        "CUSTOMER.NAME": "Mickey Mouse",
        "ETA": "09/19/2022 13:55:26",
        "ETD": "09/5/2022 13:55:26",
        "CONSOL": "Y",
        "PORT": "LKCMB"
        },
        {
        "UUID": "Y6789",
        "CUSTOMER.NAME": "Goofy",
        "ETA": "10/20/2022 13:55:26",
        "ETD": "10/5/2022 13:55:26",
        "CONSOL": "N",
        "PORT": "CNCAN"
        }
    ]
}
dtype_dict = {"UUID": "string",
              "CUSTOMER.NAME": "string",
              "ETA": "datetime64[D]",
              "ETD": "datetime64[D]",
              "CONSOL": "bool",
              "PORT": "string"
             }

### Set the computed columns data types
The data types must be set to perform the conditional logic such as comparing dates

In [147]:
import pandas as pd
import json

df_data = pd.DataFrame(jsonData['data'])
df_data = df_data.astype(dtype_dict)

print(df_data.dtypes)

UUID                     string
CUSTOMER.NAME            string
ETA              datetime64[ns]
ETD              datetime64[ns]
CONSOL                     bool
PORT                     string
dtype: object


### Completed Sheet

In [151]:
jsonCompletedData = {
    "data": [{
        "UUID": "X1234",
        "CUSTOMER.NAME": "Mickey Mouse",
        "DISPAT.DATE": "09/19/2022 13:55:26",
        "COMPLETED.DATE": "09/5/2022 13:55:26",
        "CONSOL": "Y",
        "PORT": "LKCMB"
        },
        {
        "UUID": "Y6789",
        "CUSTOMER.NAME": "Goofy",
        "DISPAT.DATE": "10/20/2022 13:55:26",
        "COMPLETED.DATE": "10/5/2022 13:55:26",
        "CONSOL": "N",
        "PORT": "CNCAN"
        }
    ]
}
completed_dtype_dict = {"UUID": "string",
              "CUSTOMER.NAME": "string",
              "DISPAT.DATE": "datetime64[D]",
              "COMPLETED.DATE": "datetime64[D]",
              "CONSOL": "bool",
              "PORT": "string"
             }

### Set completed sheet data types
The data types must be set to perform the conditional logic such as comparing dates

In [152]:
import pandas as pd
import json

df_data = pd.DataFrame(jsonCompletedData['data'])
df_data = df_data.astype(completed_dtype_dict)

print(df_data.dtypes)

UUID                      string
CUSTOMER.NAME             string
DISPAT.DATE       datetime64[ns]
COMPLETED.DATE    datetime64[ns]
CONSOL                      bool
PORT                      string
dtype: object


In [153]:
from datetime import datetime, date, timedelta
import pandas as pd
import re

cust_edi_uuid = "Y6789"
_s_given = "_data.loc[_data['UUID'] == '{cust_edi_uuid}']"
''' make a copy of the dataset with _data as the dataset name '''
_data = df_data.copy()
''' Use regex to replace the PK value to select the rows '''
_s_regex = r"{cust_edi_uuid}"
thisRegex = re.compile(_s_regex, re.IGNORECASE)
_given_mask = re.sub(_s_regex, cust_edi_uuid, _s_given)
#print(_given_mask)
''' execute the given, using, when, and action strings using python eval function '''
#print(_data.loc[_data['UUID'] == 'Y6789'])
_given_data=eval(_given_mask)
print("given data",_given_data)
_s_using = "_given_data[['UUID', 'DISPAT.DATE', 'COMPLETED.DATE', 'CONSOL', 'PORT']]"
_using_data = eval(_s_using)
print("using data",_using_data)
_s_when="_using_data['DISPAT.DATE'] < (pd.to_datetime(date.today())-timedelta(days=30))"
#_s_when = "_using_data.loc[_using_data['DISPAT.DATE'] < (pd.to_datetime(date.today())-timedelta(days=30))]"
_when_data = eval(_s_when)
print("when data",_when_data)
#_s_action = "str(_when_data['ETA'].item())+' '+str(_when_data['CONSOL'].item())+' '+str(_when_data['PORT'].item())"
_s_action = "_when_data.item()"
_action_data = eval(_s_action)
print("action data",_action_data)
''' return the computed value'''
returnVal = _action_data
print(returnVal)

given data     UUID CUSTOMER.NAME DISPAT.DATE COMPLETED.DATE  CONSOL   PORT
1  Y6789         Goofy  2022-10-20     2022-10-05    True  CNCAN
using data     UUID DISPAT.DATE COMPLETED.DATE  CONSOL   PORT
1  Y6789  2022-10-20     2022-10-05    True  CNCAN
when data 1    False
Name: DISPAT.DATE, dtype: bool
action data False
False


## Query object class and resolver 

In [154]:
class GWTInput(InputObjectType):
    given = String(name="given",
                   data=String(default_value="Select dataset rows"),
                   description="given dataset rows to use in the DSL",
                   required=False
                  )
    using = String(name="using",
                   filterCols = String(default_value="Select the columns"),
                   description="list of columns to use for the data, conditions and action",
                   required=True
                  )
    when = String(name="when",
                   conditions = String(default_value="check the conditions"),
                   description="apply the conditions to perform the action",
                   required=False
                  )
    action = String(name="action",
                   conditions = String(default_value="get the value"),
                   description="perform computation action to return the value",
                   required=True
                  )

class GWT(ObjectType):
    given = String(name="given",
                   data=String(default_value="Select dataset rows"),
                   description="given dataset rows to use in the DSL",
                   required=False
                  )
    using = String(name="using",
                   filterCols = String(default_value="Select the columns"),
                   description="list of columns to use for the data, conditions and action",
                   required=True
                  )
    when = String(name="when",
                   conditions = String(default_value="check the conditions"),
                   description="apply the conditions to for the action",
                   required=False
                  )
    action = String(name="action",
                   conditions = String(default_value="get the value"),
                   description="perform computation action to return the value",
                   required=True
                  )

class PerformAction(Mutation):
    from datetime import date
    import pandas as pd
    import re

    class Arguments:
        dsl_input = GWTInput(required=True)

    returnVal = String()
    gwt = Field(GWT)

    def mutate(self, info, dsl_input = None):

        gwt = GWT(given=dsl_input.given,
                     when=dsl_input.when,
                     using=dsl_input.using,
                     action=dsl_input.action
                    )

        cust_edi_uuid = "Y6789"

        ''' make a copy of the dataset with _data as the dataset name '''
        _data = df_data.copy()
        ''' Use regex to replace the PK value to select the rows '''
        _s_regex = r"{cust_edi_uuid}"
        thisRegex = re.compile(_s_regex, re.IGNORECASE)
        _given_mask = re.sub(_s_regex, cust_edi_uuid, gwt.given)
        
        ''' execute the given, using, when, and action strings using python eval function '''
        _given_data=eval(_given_mask)
        _using_data = eval(gwt.using)
        _when_data = eval(gwt.when)
        _action_data = eval(gwt.action)
        ''' return the computed value'''
        returnVal = _action_data

        return PerformAction(gwt=gwt, returnVal=returnVal)

class gwtActionMutation(ObjectType):

    compute_action = PerformAction.Field()

class gwtQuery(ObjectType):
    gwt = Field(GWT)
    



## Print the Schema

In [155]:
schema = Schema(query=gwtQuery, mutation=gwtActionMutation, auto_camelcase = True)
print(schema)

schema {
  query: gwtQuery
  mutation: gwtActionMutation
}

type gwtQuery {
  gwt: GWT
}

type GWT {
  """given dataset rows to use in the DSL"""
  given(data: String = "Select dataset rows"): String

  """list of columns to use for the data, conditions and action"""
  using(filterCols: String = "Select the columns"): String!

  """apply the conditions to for the action"""
  when(conditions: String = "check the conditions"): String

  """perform computation action to return the value"""
  action(conditions: String = "get the value"): String!
}

type gwtActionMutation {
  computeAction(dslInput: GWTInput!): PerformAction
}

type PerformAction {
  returnVal: String
  gwt: GWT
}

input GWTInput {
  """given dataset rows to use in the DSL"""
  given: String

  """list of columns to use for the data, conditions and action"""
  using: String!

  """apply the conditions to perform the action"""
  when: String

  """perform computation action to return the value"""
  action: String!
}


## Define the query object
This object should be generated at the time of defining the computed column and stored as a JSON. Thereafter, build the ```query_action``` with the ```mutation```

### Computed Columns

In [148]:
query_action = '''
mutation someComputedCol {
    computeAction(dslInput: 
    {
        given : "_data.loc[_data['UUID'] == '{cust_edi_uuid}']"
        using : "_given_data[['UUID', 'ETA', 'ETD', 'CONSOL', 'PORT']]"
        when : "_using_data.loc[_using_data['ETA'] > pd.to_datetime(date.today())]"
        action : "str(_when_data['ETA'].item())+' '+str(_when_data['CONSOL'].item())+' '+str(_when_data['PORT'].item())"
        })
        {
        gwt {
            given
            when
            using
            action
        }
        returnVal
    }
}
'''

### Completed Sheet

In [156]:
query_action = '''
mutation someComputedCol {
    computeAction(dslInput: 
    {
        given : "_data.loc[_data['UUID'] == '{cust_edi_uuid}']"
        using : "_given_data[['UUID', 'DISPAT.DATE', 'COMPLETED.DATE', 'CONSOL', 'PORT']]"
        when : "_using_data['DISPAT.DATE'] < (pd.to_datetime(date.today())-timedelta(days=30))"
        action : "_when_data.item()"
        })
        {
        gwt {
            given
            when
            using
            action
        }
        returnVal
    }
}
'''

## Execute the schema

In [157]:
action_result = schema.execute(query_action)
print(json.dumps(action_result.data,indent=2))

{
  "computeAction": {
    "gwt": {
      "given": "_data.loc[_data['UUID'] == '{cust_edi_uuid}']",
      "when": "_using_data['DISPAT.DATE'] < (pd.to_datetime(date.today())-timedelta(days=30))",
      "using": "_given_data[['UUID', 'DISPAT.DATE', 'COMPLETED.DATE', 'CONSOL', 'PORT']]",
      "action": "_when_data.item()"
    },
    "returnVal": "false"
  }
}
